In [1]:
%timeit range(1000)
import math
import torch
import numpy as np
from torch.autograd import Variable
from numba import jit

# Interpolation kernel
@jit
def u(l1, l2, l3, l4,a):
    x1 = abs(l1)
    x2 = abs(l2)
    x3 = abs(l3)
    x4 = abs(l4)

    y1 = 0
    if (x1 >=0) & (x1 <=1):

        y1 = (a+2)*(x1**3)-(a+3)*(x1**2)+1
    elif (x1 > 1) & (x1 <= 2):
        y1 = a*(x1**3)-(5*a)*(x1**2)+(8*a)*x1-4*a


    y2 = 0
    if (x2 >=0) & (x2 <=1):

        y2 = (a+2)*(x2**3)-(a+3)*(x2**2)+1
    elif (x2 > 1) & (x2 <= 2):
        y2 = a*(x2**3)-(5*a)*(x2**2)+(8*a)*x2-4*a


    y3 = 0
    if (x3 >=0) & (x3 <=1):

        y3 = (a+2)*(x3**3)-(a+3)*(x3**2)+1
    elif (x3 > 1) & (x3 <= 2):
        y3 = a*(x3**3)-(5*a)*(x3**2)+(8*a)*x3-4*a


    y4 = 0
    if (x4 >=0) & (x4 <=1):

        y4 = (a+2)*(x4**3)-(a+3)*(x4**2)+1
    elif (x4 > 1) & (x4 <= 2):
        y4 = a*(x4**3)-(5*a)*(x4**2)+(8*a)*x4-4*a


    return y1, y2, y3, y4

@jit
def d(mat):
    mat_l, mat_m, mat_r = mat
    d = np.dot( np.dot(mat_l, mat_m), mat_r)
    return d

@jit
def mat(h,a,i,j,c,img):
    x, y = i * h + 2 , j * h + 2

    fx = math.floor(x)
    x1 = 1 + x -fx
    x2 = x - fx
    x3 = fx + 1 - x
    x4 = fx + 2 - x

    fy = math.floor(y)
    y1 = 1 + y - fy
    y2 = y - fy
    y3 = fy + 1 - y
    y4 = fy + 2 - y

    ny1 = int(y-y1)
    ny2 = int(y-y2)
    ny3 = int(y+y3)
    ny4 = int(y+y4)
    nmx1 = int(x-x1)
    nmx2 = int(x-x2)
    npx3 = int(x+x3)
    npx4 = int(x+x4)

    mat_m = np.array([[img[0,c,ny1,nmx1],img[0,c,ny2,nmx1],img[0,c,ny3,nmx1],img[0,c,ny4,nmx1]],
                       [img[0,c,ny1,nmx2],img[0,c,ny2,nmx2],img[0,c,ny3,nmx2],img[0,c,ny4,nmx2]],
                       [img[0,c,ny1,npx3],img[0,c,ny2,npx3],img[0,c,ny3,npx3],img[0,c,ny4,npx3]],
                       [img[0,c,ny1,npx4],img[0,c,ny2,npx4],img[0,c,ny3,npx4],img[0,c,ny4,npx4]]])

    p1, p2, p3, p4 = u(y1, y2, y3, y4,a)
    

    return np.array([[u(x1, x2, x3, x4,a)]]),mat_m, np.array([[p1],[p2],[p3],[p4]])


#Paddnig
def padding(img,B,C,H,W):
    zimg = np.zeros([B,C,H+4,W+4])
    zimg[:,:C,2:H+2,2:W+2] = img
    #Pad the first/last two col and row
    zimg[:,:C,2:H+2,0:2]=img[:,:C,:,0:1]
    zimg[:,:,H+2:H+4,2:W+2]=img[:,:,H-1:H,:]
    zimg[:,:,2:H+2,W+2:W+4]=img[:,:,:,W-1:W]
    zimg[:,:C,0:2,2:W+2]=img[:,:C,0:1,:]
    #Pad the missing eight points
    zimg[0,:C,0:2,0:2]=img[0,:C,0,0]
    zimg[0,:C,H+2:H+4,0:2]=img[0,:C,H-1,0]
    zimg[0,:C,H+2:H+4,W+2:W+4]=img[0,:C,H-1,W-1]
    zimg[0,:C,0:2,W+2:W+4]=img[0,:C,0,W-1]
    return zimg

# Bicubic operation
def bicubic(img):

    # Coefficient
    img = img.cpu().detach().numpy()
    a = -0.75

    #Get image size
    B, C, H, W = img.shape

    img = padding(img,B,C,H,W)

    #Create new image
    dH = 6
    dW = 6
    dst = np.zeros([B, C, dH, dW])

    h = 1/(6/H)

    for b in range(B):
        for c in range(C):
            for j in range(dH):
                for i in range(dW):

                    dst[b, c, j, i] = d(mat(h,a,i,j,c,img))

    dst = torch.Tensor(dst)#.cuda()
    #dst = dst.type(torch.cuda.FloatTensor)
    #dst = Variable(dst, requires_grad=True).cuda()
    return dst

torchImg = torch.FloatTensor([[[[1,1,1],[2,2,2],[3,3,3]]], [[[1,1,1],[2,2,2],[3,3,3]]]])
#torchImg = torchImg.permute(0, 2,3, 1)
print('torchImg',torchImg.shape)

dst = bicubic(torchImg)
print(torch.Tensor(dst).shape)
print(dst)


249 ns ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
torchImg torch.Size([2, 1, 3, 3])
torch.Size([2, 1, 6, 6])
tensor([[[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.4062, 1.4062, 1.4062, 1.4062, 1.4062, 1.4062],
          [2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000],
          [2.5938, 2.5938, 2.5938, 2.5938, 2.5938, 2.5938],
          [3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000],
          [3.0938, 3.0938, 3.0938, 3.0938, 3.0938, 3.0938]]],


        [[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.4062, 1.4062, 1.4062, 1.4062, 1.4062, 1.4062],
          [2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000],
          [2.5938, 2.5938, 2.5938, 2.5938, 2.5938, 2.5938],
          [3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000],
          [3.0938, 3.0938, 3.0938, 3.0938, 3.0938, 3.0938]]]])


In [2]:
%timeit range(1000)
import tensorflow as tf
import torch
from torch.autograd import Variable
import numba


def bicubic(tensor, upsc_size = 6, interp = 'bicubic', align_corners=False, name=None):

    tensor = tensor.permute(0, 2, 3, 1)
    
    tfTensor = tf.convert_to_tensor(tensor)

    bicubic = tf.image.resize_bicubic(
                                            tfTensor,
                                            (upsc_size, upsc_size ),
                                            align_corners=False,
                                            name=None
                                            )
    a = tf.InteractiveSession()
    torchTensor = torch.from_numpy(bicubic.eval())
    a.close()
    torchTensor  = torchTensor.permute(0, 3, 1, 2)

    #torchTensor = torchTensor.type(torch.cuda.FloatTensor)
    #torchTensor = Variable(torchTensor, requires_grad=True).cuda()

    return torchTensor

torchImg = torch.FloatTensor([[[[1,1,1],[2,2,2],[3,3,3]]], [[[1,1,1],[2,2,2],[3,3,3]]]])

d = bicubic(torchImg)
print(d)


227 ns ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


tensor([[[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.4062, 1.4062, 1.4062, 1.4062, 1.4062, 1.4062],
          [2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000],
          [2.5938, 2.5938, 2.5938, 2.5938, 2.5938, 2.5938],
          [3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000],
          [3.0938, 3.0938, 3.0938, 3.0938, 3.0938, 3.0938]]],


        [[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.4062, 1.4062, 1.4062, 1.4062, 1.4062, 1.4062],
          [2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000],
          [2.5938, 2.5938, 2.5938, 2.5938, 2.5938, 2.5938],
          [3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000],
          [3.0938, 3.0938, 3.0938, 3.0938, 3.0938, 3.0938]]]])


In [3]:
%timeit range(1000)
import math
import sys, time
import torch
import numpy as np
import numba


# Interpolation kernel
def u(s,a):
    if (abs(s) >=0) & (abs(s) <=1):
        return (a+2)*(abs(s)**3)-(a+3)*(abs(s)**2)+1
    elif (abs(s) > 1) & (abs(s) <= 2):
        return a*(abs(s)**3)-(5*a)*(abs(s)**2)+(8*a)*abs(s)-4*a
    return 0

#Paddnig
def padding(img,B,C,H,W):
    zimg = np.zeros((B,C,H+4,W+4))
    zimg[:,:C,2:H+2,2:W+2] = img
    #Pad the first/last two col and row
    zimg[:,:C,2:H+2,0:2]=img[:,:C,:,0:1]
    zimg[:,:,H+2:H+4,2:W+2]=img[:,:,H-1:H,:]
    zimg[:,:,2:H+2,W+2:W+4]=img[:,:,:,W-1:W]
    zimg[:,:C,0:2,2:W+2]=img[:,:C,0:1,:]
    #Pad the missing eight points
    zimg[:,:C,0:2,0:2]=img[:,:C,0,0]
    zimg[:,:C,H+2:H+4,0:2]=img[:,:C,H-1,0]
    zimg[:,:C,H+2:H+4,W+2:W+4]=img[:,:C,H-1,W-1]
    zimg[:,:C,0:2,W+2:W+4]=img[:,:C,0,W-1]
    return zimg

# https://github.com/yunabe/codelab/blob/master/misc/terminal_progressbar/progress.py
def get_progressbar_str(progress):
    END = 170
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

# Bicubic operation
def bicubic(img, ratio, a):

    img = np.array(img)
    #Get image size
    B, C, H, W = img.shape
    
    print(img.shape)
    img = padding(img,B,C,H,W)
    print(img.shape)
    
    #Create new image
    dH = math.floor(H*ratio)
    dW = math.floor(W*ratio)
    dst = np.zeros((B, C, dH, dW))
    print(dst.shape)

    h = 1/ratio
    
    print("h", h)

    print('Start bicubic interpolation')
    print('It will take a little while...')
    inc = 0
    for b in range(B):
        for c in range(C):
            for j in range(dH):
                for i in range(dW):
                    x, y = i * h + 2 , j * h + 2

                    x1 = 1 + x - math.floor(x)
                    x2 = x - math.floor(x)
                    x3 = math.floor(x) + 1 - x
                    x4 = math.floor(x) + 2 - x

                    y1 = 1 + y - math.floor(y)
                    y2 = y - math.floor(y)
                    y3 = math.floor(y) + 1 - y
                    y4 = math.floor(y) + 2 - y

                    mat_l = np.matrix([[u(x1,a),u(x2,a),u(x3,a),u(x4,a)]])
                    mat_m = np.matrix([[img[0,c,int(y-y1),int(x-x1)],img[0,c,int(y-y2),int(x-x1)],img[0,c,int(y+y3),int(x-x1)],img[0,c,int(y+y4),int(x-x1)]],
                                       [img[0,c,int(y-y1),int(x-x2)],img[0,c,int(y-y2),int(x-x2)],img[0,c,int(y+y3),int(x-x2)],img[0,c,int(y+y4),int(x-x2)]],
                                       [img[0,c,int(y-y1),int(x+x3)],img[0,c,int(y-y2),int(x+x3)],img[0,c,int(y+y3),int(x+x3)],img[0,c,int(y+y4),int(x+x3)]],
                                       [img[0,c,int(y-y1),int(x+x4)],img[0,c,int(y-y2),int(x+x4)],img[0,c,int(y+y3),int(x+x4)],img[0,c,int(y+y4),int(x+x4)]]])
                    mat_r = np.matrix([[u(y1,a)],[u(y2,a)],[u(y3,a)],[u(y4,a)]])
                    dst[b, c, j, i] = np.asarray(
                                                np.dot(
                                                np.dot(mat_l, mat_m),
                                                mat_r)
                                                )

                # Print progress
                inc = inc + 1
                sys.stderr.write('\r\033[K' + get_progressbar_str(inc/(C*dH*dW)))
                sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    return dst


# Scale factor
ratio = 2
# Coefficient
a = -0.75

torchImg = torch.FloatTensor([[[[1,1,1],[2,2,2],[3,3,3]]], [[[1,1,1],[2,2,2],[3,3,3]]]])
#torchImg = torchImg.permute(0, 2,3, 1)
print('torchImg',torchImg.shape)

dst = bicubic(torchImg , ratio, a)
print(torch.Tensor(dst).shape)
print(dst)

Progress:[==========>                    ] 33.3%


225 ns ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
torchImg torch.Size([2, 1, 3, 3])
(2, 1, 3, 3)
(2, 1, 7, 7)
(2, 1, 6, 6)
h 0.5
Start bicubic interpolation
It will take a little while...
torch.Size([2, 1, 6, 6])
[[[[1.      1.      1.      1.      1.      1.     ]
   [1.40625 1.40625 1.40625 1.40625 1.40625 1.40625]
   [2.      2.      2.      2.      2.      2.     ]
   [2.59375 2.59375 2.59375 2.59375 2.59375 2.59375]
   [3.      3.      3.      3.      3.      3.     ]
   [3.09375 3.09375 3.09375 3.09375 3.09375 3.09375]]]


 [[[1.      1.      1.      1.      1.      1.     ]
   [1.40625 1.40625 1.40625 1.40625 1.40625 1.40625]
   [2.      2.      2.      2.      2.      2.     ]
   [2.59375 2.59375 2.59375 2.59375 2.59375 2.59375]
   [3.      3.      3.      3.      3.      3.     ]
   [3.09375 3.09375 3.09375 3.09375 3.09375 3.09375]]]]
